전처리

In [4]:
import pandas as pd

# 업로드된 파일 경로
file_path = 'tomato.csv'

# 파일 읽기
try:
    tomato_data = pd.read_csv(file_path)
    tomato_data_info = tomato_data.info()  # 데이터 정보 확인
    tomato_data_head = tomato_data.head()  # 데이터 첫 5행 미리보기
except Exception as e:
    tomato_data_info = str(e)
    tomato_data_head = None

tomato_data_info, tomato_data_head




<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11656 entries, 0 to 11655
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   수집일자    11656 non-null  object 
 1   건구 온도   11656 non-null  float64
 2   습구 온도   11656 non-null  float64
 3   근권      11656 non-null  float64
 4   상대 습도   11656 non-null  float64
 5   절대 습도   11656 non-null  float64
 6   이슬점     11656 non-null  float64
 7   수분 부족   11656 non-null  float64
 8   이산화탄소   11656 non-null  int64  
dtypes: float64(7), int64(1), object(1)
memory usage: 819.7+ KB


(None,
                 수집일자  건구 온도  습구 온도   근권  상대 습도  절대 습도  이슬점  수분 부족  이산화탄소
 0  1/9/2021 00:03:00    8.2    6.3  7.3   76.3    6.4  4.3    2.0   1037
 1  1/9/2021 00:08:00    8.2    6.5  7.3   77.9    6.6  4.6    1.9   1040
 2  1/9/2021 00:13:00    8.1    6.5  7.3   78.6    6.6  4.6    1.8   1041
 3  1/9/2021 00:18:00    8.1    6.6  7.4   80.9    6.8  5.1    1.6   1038
 4  1/9/2021 00:23:00    8.1    6.7  7.4   81.7    6.9  5.2    1.5   1035)

In [7]:
# 날짜 데이터를 datetime 형식으로 변환 (오류 값은 NaT로 처리)
tomato_data['수집일자'] = pd.to_datetime(tomato_data['수집일자'], errors='coerce')

# 변환되지 않은 값 확인
invalid_dates = tomato_data['수집일자'].isnull().sum()
if invalid_dates > 0:
    print(f"'{invalid_dates}'개의 수집일자 값이 변환되지 않았습니다. 데이터 확인이 필요합니다.")
# 변환에 실패한 행 출력
invalid_rows = tomato_data[tomato_data['수집일자'].isnull()]
print("잘못된 수집일자 값들:")
print(invalid_rows)


'6222'개의 수집일자 값이 변환되지 않았습니다. 데이터 확인이 필요합니다.
잘못된 수집일자 값들:
      수집일자  건구 온도  습구 온도    근권  상대 습도  절대 습도   이슬점  수분 부족  이산화탄소
3455   NaT    7.4    7.0   6.3   95.1    7.6   6.7    0.4   1064
3456   NaT    8.1    8.0   7.5   97.7    8.2   7.8    0.2   1067
3457   NaT    9.0    9.0   8.8   99.9    8.9   9.0    0.0   1072
3458   NaT    8.8    8.6   8.4   97.8    8.6   8.5    0.2   1080
3459   NaT    8.1    7.6   7.2   94.4    7.9   7.2    0.5   1083
...    ...    ...    ...   ...    ...    ...   ...    ...    ...
11651  NaT   10.9   10.7  10.7   97.9    9.7  10.6    0.2    833
11652  NaT   10.3    9.8  10.3   93.4    9.0   9.3    0.6    828
11653  NaT   10.2    9.8  10.0   94.9    9.0   9.4    0.5    825
11654  NaT   10.4   10.1  10.1   96.4    9.3   9.9    0.3    822
11655  NaT   10.6   10.4  10.3   97.2    9.5  10.2    0.3    822

[6222 rows x 9 columns]


In [5]:
# 필요한 라이브러리 불러오기
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# CSV 파일 읽기 (파일 경로를 자신의 파일 경로로 변경하세요)
file_path = 'tomato.csv'  # 예: 'C:/data/tomato.csv'
tomato_data = pd.read_csv(file_path)

# 날짜 데이터를 datetime 형식으로 변환
tomato_data['수집일자'] = pd.to_datetime(tomato_data['수집일자'])

# 품질 기준에 따른 품질 라벨링 함수
def label_quality(row):
    if 8 <= row['건구 온도'] <= 12 and row['상대 습도'] >= 90 and 800 <= row['이산화탄소'] <= 1000:
        return "좋음"
    elif 6 <= row['건구 온도'] <= 14 and 80 <= row['상대 습도'] < 90 and (600 <= row['이산화탄소'] < 800 or 1000 < row['이산화탄소'] <= 1100):
        return "보통"
    else:
        return "나쁨"

# 품질 라벨링 적용
tomato_data['품질'] = tomato_data.apply(label_quality, axis=1)

# 품질 분포 시각화
plt.figure(figsize=(8, 6))
sns.countplot(data=tomato_data, x='품질', palette='coolwarm', order=["좋음", "보통", "나쁨"])
plt.title('토마토 품질 분포', fontsize=16)
plt.xlabel('품질', fontsize=14)
plt.ylabel('개수', fontsize=14)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.show()


ValueError: time data "13/9/2021 00:00:00" doesn't match format "%m/%d/%Y %H:%M:%S", at position 3455. You might want to try:
    - passing `format` if your strings have a consistent format;
    - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
    - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.